In [1]:
import os
import shutil
import tempfile
import numpy as np
from itertools import combinations

import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from monai import transforms
from monai.apps import DecathlonDataset
from monai.config import print_config
from monai.data import DataLoader
from monai.utils import first, set_determinism
from torch.cuda.amp import GradScaler, autocast
from torch.nn import L1Loss
from tqdm import tqdm

from monai.transforms import (
    EnsureChannelFirstd,
    CenterSpatialCropd,
    Compose,
    Lambdad,
    LoadImaged,
    Resized,
    ScaleIntensityd,
)
from monai.utils import set_determinism
from torch.cuda.amp import GradScaler, autocast
from tqdm import tqdm

from generative.inferers import DiffusionInferer
from generative.networks.nets import DiffusionModelUNet
from generative.networks.schedulers import DDPMScheduler, DDIMScheduler
from monai.data import (
    CacheDataset,
    Dataset,
    DataLoader,
    LMDBDataset,
    PersistentDataset,
    decollate_batch,
)

from generative.inferers import LatentDiffusionInferer
from generative.losses import PatchAdversarialLoss, PerceptualLoss
from generative.networks.nets import AutoencoderKL, DiffusionModelUNet, PatchDiscriminator
from generative.networks.schedulers import DDPMScheduler
from generative.metrics import FIDMetric, MMDMetric, MultiScaleSSIMMetric, SSIMMetric

GPU_NUM = '6'
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"] = GPU_NUM
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
print("Current GPU:",  os.environ["CUDA_VISIBLE_DEVICES"])

import torch.multiprocessing as mp
mp.set_sharing_strategy('file_system')

device = torch.device(f"cuda:{GPU_NUM}" if torch.cuda.is_available() else "cpu") #ADDED LATER

/scratch/faculty/thompson/chirag_tmp/miniconda3/envs/myenvnew/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Current GPU: 6


In [2]:
# pip install --upgrade numpy

In [3]:
!nvidia-smi

Sun Jan 12 20:03:36 2025       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.106.00   Driver Version: 460.106.00   CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:1B:00.0 Off |                    0 |
| N/A   40C    P0    60W / 300W |   5234MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  Off  | 00000000:1C:00.0 Off |                    0 |
| N/A   

In [4]:
!gpustat

ciscogpu01               Sun Jan 12 20:03:39 2025  460.106.00
[0] Tesla V100-SXM2-32GB | 40°C,   0 % |  5234 / 32510 MB | cjagad(3023M) cjagad(1103M) cjagad(1103M)
[1] Tesla V100-SXM2-32GB | 40°C,   0 % |  7856 / 32510 MB | cjagad(7851M)
[2] Tesla V100-SXM2-32GB | 40°C,   0 % |     3 / 32510 MB |
[3] Tesla V100-SXM2-32GB | 61°C, 100 % | 23674 / 32510 MB | yqiao(23671M)
[4] Tesla V100-SXM2-32GB | 65°C, 100 % | 24364 / 32510 MB | hlan(24361M)
[5] Tesla V100-SXM2-32GB | 45°C,   2 % | 13723 / 32510 MB | cjagad(12447M) czang(1271M)
[6] Tesla V100-SXM2-32GB | 41°C,  17 % |  5296 / 32510 MB | cjagad(1695M) cjagad(1787M) cjagad(1809M)
[7] Tesla V100-SXM2-32GB | 44°C,   0 % |  5290 / 32510 MB | hlan(5287M)


In [5]:
ms_ssim = MultiScaleSSIMMetric(spatial_dims=3, data_range=1.0, kernel_size=2)
ssim = SSIMMetric(spatial_dims=3, data_range=1.0, kernel_size=2)
mmd = MMDMetric()

In [6]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        if filename.endswith(".npy"):
            img_path = os.path.join(folder, filename)
            img = np.load(img_path)
            images.append(img)
    return images

In [7]:
dataset_name = "CAMCAN"
dataset_path = "../cjagad/LDM/2D/2d_data/male/axial" 
os.environ["MONAI_DATA_DIRECTORY"]= dataset_path
directory = os.environ.get("MONAI_DATA_DIRECTORY")
root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir)

../cjagad/LDM/2D/2d_data/male/axial


In [8]:
data_transform = Compose(
    [
        LoadImaged(keys=["image"]),
        EnsureChannelFirstd(keys=["image"], channel_dim="no_channel"),
        ScaleIntensityd(keys=["image"]),
        CenterSpatialCropd(keys=["image"], roi_size=[160, 200, 160]),
        Resized(keys=["image"], spatial_size=(64, 64, 64)),
    ]
)



In [9]:
def get_data_files(root_dir):
    nifti_file_names = os.listdir(root_dir)
    folder_names = [os.path.join(
        root_dir, nifti_file_name) 
        for nifti_file_name in nifti_file_names 
        if nifti_file_name.endswith('.nii.gz') or nifti_file_name.endswith('.npy')]
    return folder_names

In [10]:
paths = get_data_files(root_dir)
train_files = [{"image": path} for path in paths]
print(len(train_files))

train_dataset = CacheDataset(
    data=train_files, transform=data_transform, cache_rate=1.0, runtime_cache="processes", copy_cache=False 
)

train_loader = DataLoader(train_dataset, batch_size=1, num_workers=2, persistent_workers=True)


314


In [11]:
male_real_images = []
for batch in train_loader:
    male_real_images.append(batch['image'])
    print(len(male_real_images))

1


KeyboardInterrupt: 

In [ ]:
male_real_images_tensor = torch.stack(male_real_images)


In [ ]:
male_syn_images = load_images_from_folder('../cjagad/LDM/2D/2D_SYN_DATA/LDM_MALE')

In [ ]:
male_syn_images = [torch.tensor(img) for img in male_syn_images]
male_syn_images_tensor = torch.stack(male_syn_images)


In [ ]:
len(male_syn_images)

In [ ]:
ms_ssim_scores = []
ssim_scores = []
mmd_scores = []

In [ ]:
with open('output_eval_metrics_real_syn_LDM 2D_male.txt', 'w') as f:
    print("LDM 2D MALE - Diversity (Real vs Synthetic) MS-SSIM and MMD \n")
    f.write("LDM 2D MALE - Diversity (Real vs Synthetic) MS-SSIM and MMD \n")
    f.flush()
    count_real = 1  
    for idx_real in range(len(male_real_images_tensor)):
        print(f'count_real: {count_real}\n')
        f.write(f'count_real: {count_real}\n')
        f.flush()
        count_syn = 1 
        for idx_synthetic in range(len(male_syn_images_tensor)):
            print(f'count_syn: {count_syn}\n')
            f.write(f'count_syn: {count_syn}\n')
            f.flush()
            
            # ms_ssim_score = ms_ssim(male_syn_images_tensor[[idx_synthetic]].squeeze(0).unsqueeze(1), male_real_images_tensor[[idx_real]].squeeze(0).squeeze(1).unsqueeze(1))#OG
            # Or alternatively, store the exact tensors we'll use and verify them:
            syn = male_syn_images_tensor[[idx_synthetic]].squeeze().unsqueeze(0).unsqueeze(1)
            real = male_real_images_tensor[[idx_real]].squeeze().unsqueeze(0).unsqueeze(1)
            
            # print("Synthetic shape:", syn.shape)
            # print("Real shape:", real.shape)
            # print("Synthetic tensor:", syn)  # This will show the actual tensor values
            # print("Real tensor:", real)      # This will show the actual tensor values
            
            # Use these exact variables
            ms_ssim_score = ms_ssim(syn, real)

            
            ssim_score = ssim(male_syn_images_tensor[[idx_synthetic]].squeeze().unsqueeze(0).unsqueeze(1), male_real_images_tensor[[idx_real]].squeeze().unsqueeze(0).unsqueeze(1))
            mmd_score = mmd(male_syn_images_tensor[[idx_synthetic]].squeeze().unsqueeze(0).unsqueeze(1), male_real_images_tensor[[idx_real]].squeeze().unsqueeze(0).unsqueeze(1))

            ms_ssim_scores.append(ms_ssim_score)
            ssim_scores.append(ssim_score)
            mmd_scores.append(mmd_score)
            count_syn += 1
        count_real += 1
    
    ms_ssim_scores_np = np.array(ms_ssim_scores)
    ssim_scores_np = np.array(ssim_scores)
    mmd_scores_np = np.array(mmd_scores)

    # np.save('ms_ssim_scores_80ae_2000ldm_500images.npy', ms_ssim_scores_np)
    # np.save('ssim_scores_80ae_2000ldm_500images.npy', ssim_scores_np)
    # np.save('mmd_scores_80ae_2000ldm_500images.npy', mmd_scores_np)
    
    ms_ssim_scores = [tensor for tensor in ms_ssim_scores]
    ssim_scores = [tensor for tensor in ssim_scores]
    mmd_scores = [torch.tensor([[tensor]]) for tensor in mmd_scores]

    ms_ssim_scores = torch.cat(ms_ssim_scores, dim=0)
    ssim_scores = torch.cat(ssim_scores, dim=0)
    mmd_scores = torch.cat(mmd_scores, dim=0)
    
    f.write(f'ms_ssim_scores: {ms_ssim_scores}\n')
    f.write(f'ssim_scores: {ssim_scores}\n')
    f.write(f'mmd_scores: {mmd_scores}\n')
    
    print(f"MS-SSIM Metric: {ms_ssim_scores.mean():.4f} +- {ms_ssim_scores.std():.4f}")
    print(f"SSIM Metric: {ssim_scores.mean():.4f} +- {ssim_scores.std():.4f}")
    print(f"MMD Metric: {mmd_scores.mean():.4f} +- {mmd_scores.std():.4f}")
    
    f.write(f"MS-SSIM Metric: {ms_ssim_scores.mean():.4f} +- {ms_ssim_scores.std():.4f}\n")
    f.flush()
    f.write(f"SSIM Metric: {ssim_scores.mean():.4f} +- {ssim_scores.std():.4f}\n")
    f.flush()
    f.write(f"MMD Metric: {mmd_scores.mean():.4f} +- {mmd_scores.std():.4f}\n")
    f.flush()

In [ ]:
ms_ssim_scores = np.load('ms_ssim_scores.npy', allow_pickle=True)

In [ ]:
mmd_scores = np.load('mmd_scores.npy', allow_pickle=True)

In [ ]:
ssim_scores = np.load('ssim_scores.npy', allow_pickle=True)

In [ ]:
len(ms_ssim_scores)

In [ ]:
len(mmd_scores)

In [ ]:
ssim_scores

In [ ]:
ms_ssim_scores

In [ ]:
mmd_scores

In [ ]:
ms_ssim_scores = [tensor for tensor in ms_ssim_scores]
ms_ssim_scores

In [ ]:
ms_ssim_scores = torch.cat(ms_ssim_scores, dim=0)

In [ ]:
mmd_scores = [torch.tensor([[tensor]]) for tensor in mmd_scores]
mmd_scores

In [ ]:
mmd_scores = torch.cat(mmd_scores, dim=0)
mmd_scores

In [ ]:
print(f"MS-SSIM Metric: {ms_ssim_scores.mean():.4f} +- {ms_ssim_scores.std():.4f}")
print(f"MMD Metric: {mmd_scores.mean():.4f} +- {mmd_scores.std():.4f}")

IOStream.flush timed out
